In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds
import os
import pandas as pd 
import numpy as np
import cv2
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
def dataframe_to_list(df):
    x = df.to_numpy()
    li = []
    for i in range(x.shape[0]):
        li.append([float(j) for j in x[i][0].split(" ")])
    return li
        

In [ ]:
def datasetprep(n=5):
    train_dataset_list = []
    train_label_list = []
    for i in range(n):
        df = pd.read_csv(r'C://Users/Christian/Desktop/calib_challenge/calib_challenge/labeled/' + str(i) + ".txt",delimiter="\t",header=None)
        label_list = df.values.tolist()
        train_label_list += label_list
        cap = cv2.VideoCapture(r"C://Users/Christian/Desktop/calib_challenge/calib_challenge/labeled/"+str(i)+".hevc")
        if(cap.isOpened()==False):
            print("Error Opening the video stream or file")
        while cap.isOpened():
            ret, frame = cap.read()
            
            # if frame is read correctly ret is True
            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break
            gray = frame #cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('frame', gray)
            #print((frame.shape))
            if cv2.waitKey(1) == ord('q'):
                break
                
            train_dataset_list.append(frame)
        cap.release()
        cv2.destroyAllWindows()
        
    return train_dataset_list, train_label_list

In [ ]:
train_dataset_list, train_label_list = datasetprep()

In [ ]:
train_dataset_list = np.array(train_dataset_list)
train_label_list = np.array(train_label_list)

In [ ]:
train_dataset_list.shape

In [ ]:
train_label_list.shape

In [ ]:
shape = train_dataset_list[0,:,:,:].shape

In [ ]:
IMAGE_WIDTH = 874
IMAGE_HEIGHT = 1164
IMG_SHAPE = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)
res_net = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)

In [ ]:
IMAGE_WIDTH = 874
IMAGE_HEIGHT = 1164
IMG_SHAPE = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)
vgg_net = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)

In [ ]:
from tensorflow.keras.optimizers import Adam

def ModelCreator(base_model,lr=0.001):
    global_average_layer = layers.GlobalAveragePooling2D()

    output_layer = layers.Dense(5, activation='relu')

    tl_model = tf.keras.Sequential([
        base_model,
        global_average_layer,
        layers.BatchNormalization(),
        output_layer
    ])

    tf.keras.optimizers.Adam(
        learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='Adam')
    tl_model.summary()
    tl_model.compile(loss='mean_squared_error', optimizer='Adam', metrics=["mean_squared_error"])
    return tl_model

In [ ]:
def training_model(tl_model,X_TRAIN,Y_TRAIN,batch_size=15,epoch=5):
    print(".... Result.....")
    history = tl_model.fit(X_TRAIN,Y_TRAIN,batch_size=batch_size, epochs=epoch, verbose=2, validation_split = 0.50)
    return history

In [ ]:
possible_base_models= [vgg_net,res_net]


In [ ]:
possible_models = []
learnRate = [0.1,0.01,0.001,0.0001]
for i in possible_base_models:
    for j in learnRate:
        model = ModelCreator(base_model=i,lr=j)
        possible_models.append(model)

In [ ]:

flat_dataset = train_dataset_list.flatten()
flat_label = train_label_list.flatten()

is_string_dataset = np.array([isinstance,(x, str)for x in flat_dataset])
is_string_label = np.array([isinstance,(x, str) for x in flat_label])

if is_string_dataset():
    print('String found', np.where(is_string_dataset))

if is_string_label():
    print('String found', np.where(is_string_label))

In [ ]:
######### pick up here to check whats wrong w the np.array stuff

flat_dataset = train_dataset_list.flatten()
flat_label = train_label_list.flatten()

vfunc = np.vectorize(lambda x: isinstance(x, str))

is_string_dataset = vfunc(flat_dataset)
is_string_label = vfunc(flat_label)

In [ ]:
for i in possible_models:
    training_model(i,train_dataset_list,train_label_list,batch_size=2,epoch=5)